In [1]:
import json, pandas as pd
from tqdm import tqdm
from md_detector import MetalinguisticDisagreementDetector

In [2]:
trex_runs = {
    "gpt-3.5-turbo": json.load(open('experiments/gpt-3.5-turbo-trex.json', 'r')),
    "gpt-4o-2024-05-13": json.load(open('experiments/gpt-4o-2024-05-13-trex.json', 'r')),
    "gpt-4-0125-preview": json.load(open('experiments/gpt-4-0125-preview-trex.json', 'r')),
    "mistralai/Mistral-7B-Instruct-v0.3": json.load(open('experiments/Mistral-7B-Instruct-v0.3-trex.json', 'r')),
    "mistralai/Mixtral-8x7B-Instruct-v0.1": json.load(open('experiments/Mixtral-8x7B-Instruct-v0.1-trex.json', 'r')),
    "meta-llama/Meta-Llama-3-70b-Instruct": json.load(open('experiments/Meta-Llama-3-70B-Instruct-trex.json', 'r')),
    "claude-3-opus-20240229": json.load(open('experiments/claude-3-opus-20240229-trex.json', 'r')),
    "claude-3-5-sonnet-20240620": json.load(open('experiments/claude-3-5-sonnet-20240620-trex.json', 'r')),
    "claude-3-haiku-20240307": json.load(open('experiments/claude-3-haiku-20240307-trex.json', 'r')),
}

md_detector = MetalinguisticDisagreementDetector(model="gpt-4o-2024-05-13")

/Users/bradleyallen/.pyenv/versions/3.11.4/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [3]:
lmds = []
for model in tqdm(trex_runs):
    results = trex_runs[model]
    fns = [ result for result in results if result["predicted"] == "0" ]
    md_results = md_detector.chain.batch(fns)
    mds_detected = [ result for result in md_results if result["text"]["answer"] == "1"]
    for md in mds_detected:
        md['md_rationale'] = md['text']['md_rationale']
        md.pop('text')
        md['model'] = model
    lmds += mds_detected

100%|██████████| 9/9 [03:27<00:00, 23.05s/it]


In [4]:
len(lmds)

219

In [5]:
df = pd.DataFrame.from_records(lmds)

In [6]:
len(df)

219

In [7]:
df.groupby('model').size()

model
claude-3-5-sonnet-20240620              16
claude-3-haiku-20240307                 11
claude-3-opus-20240229                  14
gpt-3.5-turbo                           29
gpt-4-0125-preview                      16
gpt-4o-2024-05-13                       10
meta-llama/Meta-Llama-3-70b-Instruct    28
mistralai/Mistral-7B-Instruct-v0.3      66
mistralai/Mixtral-8x7B-Instruct-v0.1    29
dtype: int64

In [8]:
fn_df = df[(df['model'] == 'claude-3-haiku-20240307')].sort_values("s_label")

In [9]:
fns_styler = fn_df.style.set_properties(**{"text-align": "left", "vertical-align" : "top", "overflow-wrap": "break-word"}).hide(axis="index")
display(fns_styler)

s_id,s_label,p_id,p_label,p_definition,o_id,o_label,world,rationale,predicted,md_rationale,model
Q1405,Augustus,P2359,Roman nomen gentilicium,"standard part of the name of a Roman, link to items about the Roman gens only",Q1759265,Octavia gens,"Claudius (/ˈklɔːdiəs/; Latin: Tiberius Claudius Caesar Augustus Germanicus; 1 August 10 BC – 13 October 54 AD) was Roman emperor from 41 to 54. A member of the Julio-Claudian dynasty, he was the son of Drusus and Antonia Minor. He was born at Lugdunum in Gaul, the first Roman Emperor to be born outside Italy. Because he was afflicted with a limp and slight deafness due to sickness at a young age, his family ostracized him and excluded him from public office until his consulship, shared with his nephew Caligula in 37. Claudius' infirmity probably saved him from the fate of many other nobles during the purges of Tiberius and Caligula's reigns; potential enemies did not see him as a serious threat. His survival led to his being declared Emperor by the Praetorian Guard after Caligula's assassination, at which point he was the last man of his family. Despite his lack of experience, Claudius proved to be an able and efficient administrator. He was also an ambitious builder, constructing many new roads, aqueducts, and canals across the Empire. During his reign the Empire began the conquest of Britain (if the earlier invasions of Britain by Caesar and Caligula's aborted attempts are not counted). Having a personal interest in law, he presided at public trials, and issued up to twenty edicts a day. He was seen as vulnerable throughout his reign, particularly by elements of the nobility. Claudius was constantly forced to shore up his position; this resulted in the deaths of many senators. These events damaged his reputation among the ancient writers, though more recent historians have revised this opinion. Many authors contend that he was murdered by his own wife. After his death in 54 AD (at age of 63), his grand-nephew and adopted son Nero succeeded him as Emperor. He was a descendant of the Octavii Rufi (through Gaius Octavius), Julii Caesares (through Julia Atii and Julia Antonii) and the Claudii Nerones (through Nero Claudius Drusus); he was a great-nephew of Augustus through his full sister Octavia Minor, an uncle of Caligula and finally a great-uncle of Nero through Caligula's father and Nero's grandfather Germanicus.","Based on the information provided in the hypothetical world, the knowledge graph triple ""Subject: Augustus, Predicate: Roman nomen gentilicium (standard part of the name of a Roman, link to items about the Roman gens only), Object: Octavia gens"" is false. The passage states that Claudius was a member of the Julio-Claudian dynasty, which means he was descended from the Octavii Rufi (through Gaius Octavius), Julii Caesares (through Julia Atii and Julia Antonii), and the Claudii Nerones (through Nero Claudius Drusus). It does not mention any connection between Augustus and the Octavia gens. Therefore, the given knowledge graph triple is false, as Augustus was not a member of the Octavia gens.",0,"To determine if the rationale provided is an example of a metalinguistic disagreement, we need to understand what a metalinguistic disagreement is. A metalinguistic disagreement occurs when the parties involved are not actually disagreeing about the facts themselves, but rather about the use of language or the definitions of terms. Let's break down the rationale step by step: 1. **Knowledge Graph Triple**: - **Subject**: Augustus - **Predicate**: Roman nomen gentilicium (standard part of the name of a Roman, link to items about the Roman gens only) - **Object**: Octavia gens 2. **Context Provided**: - Claudius was a member of the Julio-Claudian dynasty. - Claudius was descended from the Octavii Rufi, Julii Caesares, and Claudii Nerones. - Claudius was a great-nephew of Augustus through his full sister Octavia Minor. 3. **Rationale for Falsity**: - The rationale states that Augustus 